## Normalized market with sizing effect

In [1]:
from model_builder import model_builder
from data_gen import simulate_private_equity_cashflows
import numpy as np

In [2]:
cashflows = simulate_private_equity_cashflows()

In [3]:
# Normalize the cashflows of each fund in the market

def fund_weights(cashflows):
    funds = cashflows['FundID'].unique() 
    weights = {}
    for fund in funds:
        fund_cashflows = cashflows[cashflows['FundID'] == fund]
        weights[fund] = -fund_cashflows['Cashflow'].sum()
    return weights

cashflows_contr = cashflows[cashflows['Cashflow'] < 0]
eq_cashflows_contr = fund_weights(cashflows_contr)

def eq_calc(row):
    fund = row['FundID']
    cf = row['Cashflow']
    
    return cf / eq_cashflows_contr[fund]

# Apply the function row-wise
cashflows['eq_cashflow'] = cashflows.apply(eq_calc, axis=1)

# Drop and rename
cashflows = cashflows.drop(columns=['Cashflow']).rename(columns={'eq_cashflow': 'Cashflow'})

cashflows

,FundID,VintageYear,Strategy,Geography,FundQuality,Quarter,date,Cashflow
0,0,1996,Buyout,North America,Bad,0,1996-03-31,-0.229863
1,0,1996,Buyout,North America,Bad,1,1996-06-30,0.000000
2,0,1996,Buyout,North America,Bad,2,1996-09-30,-0.134382
3,0,1996,Buyout,North America,Bad,3,1996-12-31,0.000000
4,0,1996,Buyout,North America,Bad,4,1997-03-31,-0.117993
...,...,...,...,...,...,...,...,...
118887,2399,2001,Venture,Rest of World,Bad,39,2010-12-31,0.000000
118888,2399,2001,Venture,Rest of World,Bad,40,2011-03-31,0.000000
118889,2399,2001,Venture,Rest of World,Bad,41,2011-06-30,0.062329
118890,2399,2001,Venture,Rest of World,Bad,42,2011-09-30,0.094020


In [4]:
def portfolio_cashflows(cashflows, portfolio_size):
    # Randomly select fund IDs
    selected_funds = np.random.choice(cashflows['FundID'].unique(), size=portfolio_size, replace=False)
    
    # Create a random size adjustment for each selected fund
    size_adjustments = {fund: np.random.uniform(0.5, 2.0) for fund in selected_funds}
    
    # Filter the portfolio
    portfolio = cashflows[cashflows['FundID'].isin(selected_funds)].copy()
    
    # Apply the size adjustment
    portfolio['Cashflow'] = portfolio.apply(lambda row: row['Cashflow'] * size_adjustments[row['FundID']], axis=1)
    
    return portfolio

In [5]:
port_cfs = portfolio_cashflows(cashflows, 200)
port_cfs = port_cfs.copy()
model = model_builder(cashflows, port_cfs)

### Monte-carlo simulation

In [6]:
market_model = model_builder(cashflows, cashflows)
market_model

({'Portfolio MOIC': 1.8659841367141872,
  'Market MOIC': 1.8659841367141872,
  'Timing Alpha MOIC': -2.220446049250313e-16,
  'Strategy Alpha MOIC': -2.220446049250313e-16,
  'Geography Alpha MOIC': -2.220446049250313e-16,
  'Sizing Alpha MOIC': 0.0,
  'Residual Alpha MOIC': 6.661338147750939e-16},
 {'Portfolio IRR': 0.10700364847177736,
  'Market IRR': 0.10700364847177736,
  'Timing Alpha IRR': 0.0,
  'Strategy Alpha IRR': -4.163336342344337e-17,
  'Geography Alpha IRR': -2.7755575615628914e-17,
  'Sizing Alpha IRR': 0.0,
  'Residual Alpha IRR': 6.938893903907228e-17})

In [7]:
number_of_simulations = 5
fund_size = 200
moic_results = []
irr_results = []

for i in range(number_of_simulations):
    port_cfs = portfolio_cashflows(cashflows, fund_size)
    port_cfs = port_cfs.copy()
    model = model_builder(cashflows, port_cfs)
    moic_results.append(model[0])
    irr_results.append(model[1])


In [8]:
def average_dicts(dict_list):
    """Compute the average of a list of dictionaries."""
    keys = dict_list[0].keys()
    avg_dict = {}
    for key in keys:
        avg_dict[key] = sum(d[key] for d in dict_list) / len(dict_list)
    return avg_dict

In [9]:
avg_moic_decomposition = average_dicts(moic_results)
avg_irr_decomposition = average_dicts(irr_results)

In [10]:
avg_irr_decomposition

{'Portfolio IRR': 0.11035098987686284,
 'Market IRR': 0.10700364847177737,
 'Timing Alpha IRR': 0.0007649452191956591,
 'Strategy Alpha IRR': -9.610702983238462e-05,
 'Geography Alpha IRR': -6.550131895619038e-05,
 'Sizing Alpha IRR': -0.00024422702795752206,
 'Residual Alpha IRR': 0.002988231562635929}

In [11]:
avg_moic_decomposition

{'Portfolio MOIC': 1.9097955191697469,
 'Market MOIC': 1.8659841367141872,
 'Timing Alpha MOIC': 0.012874058027185465,
 'Strategy Alpha MOIC': -0.0012114963470847063,
 'Geography Alpha MOIC': -0.004609177145366905,
 'Sizing Alpha MOIC': 0.00066120015396951,
 'Residual Alpha MOIC': 0.03609679776685652}

## Analyze the individual effect

### Strategy effect

In [12]:
venture_ids = cashflows[cashflows['Strategy'] == 'Venture']['FundID'].unique()
buyout_ids = cashflows[cashflows['Strategy'] == 'Buyout']['FundID'].unique()
venture_ids = np.append(venture_ids,buyout_ids[0]) # we need at least one venture fund to make the model work
buyout_ids = np.append(buyout_ids,venture_ids[0]) # we need at least one buyout fund to make the model work

In [13]:
def portfolio_cashflows_by_id(cashflows, ids):

    # Create a random size adjustment for each selected fund
    size_adjustments = {fund: np.random.uniform(0.5, 2.0) for fund in ids}
    
    # Filter the portfolio
    portfolio = cashflows[cashflows['FundID'].isin(ids)].copy()
    
    # Apply the size adjustment
    portfolio['Cashflow'] = portfolio.apply(lambda row: row['Cashflow'] * size_adjustments[row['FundID']], axis=1)
    
    return portfolio

In [14]:
port_cfs_venture = portfolio_cashflows_by_id(cashflows, venture_ids)
port_cfs_venture = port_cfs_venture.copy()
port_cfs_buyout = portfolio_cashflows_by_id(cashflows, buyout_ids)
port_cfs_buyout = port_cfs_buyout.copy()

In [15]:
model_venture = model_builder(cashflows, port_cfs_venture)
model_venture

({'Portfolio MOIC': 2.1307220106786042,
  'Market MOIC': 1.8659841367141872,
  'Timing Alpha MOIC': 0.010459444250878969,
  'Strategy Alpha MOIC': 0.25099844522386494,
  'Geography Alpha MOIC': 0.008123367401277726,
  'Sizing Alpha MOIC': 0.005894767038003401,
  'Residual Alpha MOIC': -0.010738149949607978},
 {'Portfolio IRR': 0.13358174995709707,
  'Market IRR': 0.10700364847177736,
  'Timing Alpha IRR': 0.0012034586111456247,
  'Strategy Alpha IRR': 0.025586633442360937,
  'Geography Alpha IRR': 0.0012066377430515474,
  'Sizing Alpha IRR': 0.0003807221550401474,
  'Residual Alpha IRR': -0.0017993504662785487})

In [16]:
model_buyout = model_builder(cashflows, port_cfs_buyout)
model_buyout

({'Portfolio MOIC': 1.6189291654167235,
  'Market MOIC': 1.8659841367141872,
  'Timing Alpha MOIC': -0.011787586168852338,
  'Strategy Alpha MOIC': -0.24581104975443746,
  'Geography Alpha MOIC': -0.004081781684751817,
  'Sizing Alpha MOIC': 0.0004592851960487643,
  'Residual Alpha MOIC': 0.014166161114529174},
 {'Portfolio IRR': 0.07968127078883039,
  'Market IRR': 0.10700364847177736,
  'Timing Alpha IRR': -0.0012396441896546928,
  'Strategy Alpha IRR': -0.02789016954587087,
  'Geography Alpha IRR': -0.0007219630788295461,
  'Sizing Alpha IRR': 0.0006115657903138366,
  'Residual Alpha IRR': 0.0019178333410942994})

### Geo effect

In [17]:
america_ids = cashflows[cashflows['Geography'] == 'North America']['FundID'].unique()
row_ids = cashflows[cashflows['Geography'] == 'Rest of World']['FundID'].unique()
america_ids = np.append(america_ids,row_ids[0]) # we need at least one venture fund to make the model work
row_ids = np.append(row_ids,america_ids[0]) # we need at least one buyout fund to make the model work

In [18]:
port_cfs_america = portfolio_cashflows_by_id(cashflows, america_ids)
port_cfs_america = port_cfs_america.copy()
port_cfs_row = portfolio_cashflows_by_id(cashflows, row_ids)
port_cfs_row = port_cfs_row.copy()

In [19]:
model_america = model_builder(cashflows, port_cfs_america)
model_america

({'Portfolio MOIC': 2.215339430612706,
  'Market MOIC': 1.8659841367141872,
  'Timing Alpha MOIC': 0.009530588797556971,
  'Strategy Alpha MOIC': 0.0024289278883782295,
  'Geography Alpha MOIC': 0.3533299286041145,
  'Sizing Alpha MOIC': -0.010455325026692552,
  'Residual Alpha MOIC': -0.005478826364838207},
 {'Portfolio IRR': 0.14249016136244627,
  'Market IRR': 0.10700364847177736,
  'Timing Alpha IRR': 0.0010261438375228082,
  'Strategy Alpha IRR': 0.0008571385675121418,
  'Geography Alpha IRR': 0.035819313914618275,
  'Sizing Alpha IRR': -0.0006686892940592215,
  'Residual Alpha IRR': -0.0015473941349250925})

In [20]:
model_row = model_builder(cashflows, port_cfs_row)
model_row

({'Portfolio MOIC': 1.5064994211998135,
  'Market MOIC': 1.8659841367141872,
  'Timing Alpha MOIC': -0.0065839391001547565,
  'Strategy Alpha MOIC': -0.00906474950364089,
  'Geography Alpha MOIC': -0.3704222184124326,
  'Sizing Alpha MOIC': 0.014759923192148028,
  'Residual Alpha MOIC': 0.011826268309706567},
 {'Portfolio IRR': 0.0675333212756637,
  'Market IRR': 0.10700364847177736,
  'Timing Alpha IRR': -0.0003953563713513103,
  'Strategy Alpha IRR': -0.0013467959584618772,
  'Geography Alpha IRR': -0.041015109514384626,
  'Sizing Alpha IRR': 0.0020024778103824165,
  'Residual Alpha IRR': 0.0012844568377017307})

### Fund quality

In [21]:
good_ids = cashflows[cashflows['FundQuality'] == 'Good']['FundID'].unique()
bad_ids = cashflows[cashflows['FundQuality'] == 'Bad']['FundID'].unique()
good_ids = np.append(good_ids,bad_ids[0]) # we need at least one venture fund to make the model work
bad_ids = np.append(bad_ids,good_ids[0]) # we need at least one buyout fund to make the model work

In [22]:
port_cfs_good = portfolio_cashflows_by_id(cashflows, good_ids)
port_cfs_good = port_cfs_good.copy()    
port_cfs_bad = portfolio_cashflows_by_id(cashflows, bad_ids)
port_cfs_bad = port_cfs_bad.copy()

In [23]:
model_good = model_builder(cashflows, port_cfs_good)
model_good

({'Portfolio MOIC': 2.4357908818174066,
  'Market MOIC': 1.8659841367141872,
  'Timing Alpha MOIC': 0.008844053333528334,
  'Strategy Alpha MOIC': 0.010161260515637593,
  'Geography Alpha MOIC': -0.00211844841891784,
  'Sizing Alpha MOIC': 0.007101389522495172,
  'Residual Alpha MOIC': 0.5458184901504761},
 {'Portfolio IRR': 0.15877857117173935,
  'Market IRR': 0.10700364847177736,
  'Timing Alpha IRR': 0.00014590322107685405,
  'Strategy Alpha IRR': 0.001526942697399683,
  'Geography Alpha IRR': -0.00062567239587763,
  'Sizing Alpha IRR': -0.000443191369215834,
  'Residual Alpha IRR': 0.05117094054657892})

In [24]:
model_bad = model_builder(cashflows, port_cfs_bad)
model_bad

({'Portfolio MOIC': 1.3062081343651262,
  'Market MOIC': 1.8659841367141872,
  'Timing Alpha MOIC': -0.0033119021663710058,
  'Strategy Alpha MOIC': -0.015104040933716645,
  'Geography Alpha MOIC': 0.0010317089595899276,
  'Sizing Alpha MOIC': -0.004514611813349223,
  'Residual Alpha MOIC': -0.537877156395214},
 {'Portfolio IRR': 0.042978868084557725,
  'Market IRR': 0.10700364847177736,
  'Timing Alpha IRR': -5.856616120070157e-06,
  'Strategy Alpha IRR': -0.0021387741463764626,
  'Geography Alpha IRR': -4.1441670252384544e-07,
  'Sizing Alpha IRR': -0.0005533855618239311,
  'Residual Alpha IRR': -0.06132634964619665})

### Vintage effect

In [32]:
vintage_ids = {}
for vint in cashflows['VintageYear'].unique():
    vint_ids = cashflows[cashflows['VintageYear'] == vint]['FundID'].unique()
    vintage_ids[vint] = vint_ids

In [33]:
import random
def pick_one_per_vintage(fund_dict):
    selected_ids = []

    for vintage, fund_ids in fund_dict.items():
        # Randomly pick one fund from this vintage
        fund_ids = list(fund_ids)  # ensure it's a list
        selected_id = random.choice(fund_ids)
        selected_ids.append(selected_id)

    return np.array(selected_ids)

one_from_each = pick_one_per_vintage(vintage_ids)

In [34]:
good_vintage_ids = cashflows[cashflows['VintageYear'] % 2 == 1]['FundID'].unique()
bad_vintage_ids = cashflows[cashflows['VintageYear'] % 2 == 0]['FundID'].unique()
good_vintage_ids = np.append(good_vintage_ids,one_from_each) # we need at least one venture fund to make the model work
bad_vintage_ids = np.append(bad_vintage_ids,one_from_each) # we need at least one buyout fund to make the model work

In [35]:
port_cfs_good_vintage = portfolio_cashflows_by_id(cashflows, good_vintage_ids)
port_cfs_good_vintage = port_cfs_good_vintage.copy()
port_cfs_bad_vintage = portfolio_cashflows_by_id(cashflows, bad_vintage_ids)
port_cfs_bad_vintage = port_cfs_bad_vintage.copy()

In [36]:
model_good_vintage = model_builder(cashflows, port_cfs_good_vintage)
model_good_vintage

({'Portfolio MOIC': 2.1753993778962935,
  'Market MOIC': 1.8659841367141872,
  'Timing Alpha MOIC': 0.3065413954400844,
  'Strategy Alpha MOIC': -0.00796869089632457,
  'Geography Alpha MOIC': -0.07389514749690496,
  'Sizing Alpha MOIC': 0.0018487013197843183,
  'Residual Alpha MOIC': 0.08288898281546708},
 {'Portfolio IRR': 0.1381494495053371,
  'Market IRR': 0.10700364847177736,
  'Timing Alpha IRR': 0.030376983997692744,
  'Strategy Alpha IRR': -0.00292608712368618,
  'Geography Alpha IRR': -0.005567584225425734,
  'Sizing Alpha IRR': 0.0004958585013156436,
  'Residual Alpha IRR': 0.00876662988366328})

In [37]:
model_bad_vintage = model_builder(cashflows, port_cfs_bad_vintage)
model_bad_vintage

({'Portfolio MOIC': 1.5630881556325384,
  'Market MOIC': 1.8659841367141872,
  'Timing Alpha MOIC': -0.28195900909919946,
  'Strategy Alpha MOIC': 0.02408910228562644,
  'Geography Alpha MOIC': 0.09591753007544779,
  'Sizing Alpha MOIC': -0.017313241158406267,
  'Residual Alpha MOIC': -0.12363036318511722},
 {'Portfolio IRR': 0.07580988520990174,
  'Market IRR': 0.10700364847177736,
  'Timing Alpha IRR': -0.028657949157842977,
  'Strategy Alpha IRR': -0.0009477553443312525,
  'Geography Alpha IRR': 0.0157105633793784,
  'Sizing Alpha IRR': -0.0019092389997017595,
  'Residual Alpha IRR': -0.01538938313937803})